# ETL-pipelines in Azure Databricks
## Module 5: Data Laden en Opslag in Azure
In deze module leren we hoe we getransformeerde data kunnen **opslaan, exporteren en optimaliseren** in Azure Databricks.

**We behandelen:**
- Opslaan van data in Delta Lake (Brons, Zilver, Goud)
- Exporteren naar Parquet en CSV
- Laden van data in een Azure SQL Database
- CRUD-operaties in Delta Lake
- Optimaliseren van Delta Lake opslag

## Stap 1: Start een SparkSession
We beginnen met het opzetten van een SparkSession, de kern van Apache Spark.

In [ ]:
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.appName("ETL_Declaraties").getOrCreate()

## Stap 2: Laad de declaraties dataset
We laden de dataset en verkennen de gegevens.

In [ ]:
# Laad declaraties dataset
df_declaraties = spark.read.csv("/mnt/data/declaraties_dataset.csv", header=True, inferSchema=True)
df_declaraties.show(5)

## Stap 3: Opslaan van data in Delta Lake
We slaan de ruwe declaraties op in de **Brons-laag**.

In [ ]:
df_declaraties.write.format("delta").mode("overwrite").save("/mnt/data/brons/declaraties")

## Stap 4: Opslaan van opgeschoonde data in de Zilver-laag
We filteren de declaraties en slaan de opgeschoonde data op.

In [ ]:
df_clean = df_declaraties.filter("Bedrag >= 10 AND Bedrag <= 500")
df_clean.write.format("delta").mode("overwrite").save("/mnt/data/zilver/declaraties")

## Stap 5: Opslaan van geaggregeerde data in de Goud-laag
We groeperen de declaraties per zorgtype en slaan het totaalbedrag op.

In [ ]:
df_goud = df_clean.groupBy("Zorgtype").sum("Bedrag")
df_goud.write.format("delta").mode("overwrite").save("/mnt/data/goud/declaraties_per_zorgtype")

## Stap 6: Exporteren naar Parquet en CSV
Soms willen we data exporteren voor externe analyses.

In [ ]:
# Export naar Parquet
df_goud.write.format("parquet").mode("overwrite").save("/mnt/data/exports/declaraties.parquet")

# Export naar CSV
df_goud.write.format("csv").mode("overwrite").option("header", "true").save("/mnt/data/exports/declaraties.csv")

## Stap 7: Laden van data in Azure SQL Database
We laden de declaraties in een Azure SQL Database.

In [ ]:
jdbc_url = "jdbc:sqlserver://<jouw-sql-server>.database.windows.net:1433;database=<jouw-database>"
properties = {"user": "<jouw-gebruikersnaam>", "password": "<jouw-wachtwoord>", "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}

df_goud.write.jdbc(url=jdbc_url, table="DeclaratiesPerZorgtype", mode="overwrite", properties=properties)

## Stap 8: CRUD-operaties in Delta Lake
We voeren een update- en delete-operatie uit.

In [ ]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/mnt/data/goud/declaraties_per_zorgtype")
delta_table.update(condition="Zorgtype = 'Tandarts'", set={"sum(Bedrag)": "sum(Bedrag) * 1.05"})
delta_table.delete("sum(Bedrag) < 20")

## Stap 9: Data optimaliseren in Delta Lake
We verbeteren prestaties met OPTIMIZE en VACUUM.

In [ ]:
spark.sql("OPTIMIZE delta.`/mnt/data/goud/declaraties_per_zorgtype`")
spark.sql("VACUUM delta.`/mnt/data/goud/declaraties_per_zorgtype` RETAIN 7 HOURS")

## Oefeningen
1. **Sla de verzekerden dataset op in de Brons-, Zilver- en Goud-laag.**
2. **Exporteer de verzekerden dataset als Parquet-bestand.**
3. **Voeg een nieuwe declaratie toe aan de Delta Lake dataset.**